In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [6]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths  = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
#                or path.count('_DT-')
               or path.count('ProductCD_t')
#                or path.count('D')
#                or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.count('uid')
               )
               and not path.count('uid')
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
#                or path.count('_DT-')
               or path.count('ProductCD_t')
#                or path.count('D')
#                or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.count('uid')
               )
               and not path.count('uid')
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]
del df_train, df_test
gc.collect()

21

In [7]:
#========================================================================
# V同士のdiff, ratio
# 類似してるCでまとめる
#========================================================================
from itertools import combinations

combi = [
['V156', 'V314']
,['V258', 'V165']
,['V258', 'V267']
,['V258', 'V78']
,['V283', 'V314']
,['V283', 'V91']
,['V294', 'V307']
,['V294', 'V317']
,['V307', 'V310']
,['V308', 'V29']
,['V308', 'V70']
,['V308', 'V91']
,['V317', 'V310']
,['V45', 'V37']
,['V45', 'V67']
,['V49', 'V38']
,['V70', 'V128']
,['V70', 'V131']
,['V70', 'V312']
,['V87', 'V78']
,['V91', 'V131']
]

# cols_V = [col for col in cols_V]

# combi_V = combinations(cols_V, 2)
# for (f1, f2) in tqdm(list(combi_V)):
for (f1, f2) in tqdm(combi):
#     data[f'{f1}-{f2}__diff'] = data[f1] - data[f2]
    data[f'{f1}-{f2}__ratio'] = data[f1] / (data[f2]+1)

100%|██████████| 21/21 [00:00<00:00, 85.07it/s]


In [14]:
#========================================================================
# ProductCDあたりのV
#========================================================================
prefix = '512'
dir_save = 'check_trush'
cols_feature = [col for col in data.columns if col.count('__diff') or col.count('__ratio')]
cols_pcd = data['ProductCD'].unique()

# def parallel_save(df, col):
df = data
for col in cols_feature:
    for pcd in cols_pcd:
        feature_name = f'{col}__ProductCD-{pcd}'
        df[feature_name] = np.nan
        df.loc[df['ProductCD'].isin([pcd]), feature_name] = df.loc[df['ProductCD'].isin([pcd]), col]
        
        train = df.iloc[:train_length]
        test  = df.iloc[train_length:]
        
        if train[feature_name].value_counts().shape[0]<=1:
            continue
        elif test[feature_name].value_counts().shape[0]<=1:
            continue

        save_feature(train[[feature_name]], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(test[ [feature_name]], prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)
        
# for i in range(1, 40, 1):
#     Parallel(60)([delayed(parallel_save)(data[[col, 'ProductCD']], col)
#                   for col in cols_feature[i*60:60*(i+1)]])

(506691,) | V91-V131__ratio__ProductCD-W
